<a href="https://colab.research.google.com/github/David-Sanchez-Simoni/DataCordobes/blob/master/Neighbourhoods_clustering_toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Segmenting and Clustering Neighborhoods in Toronto Project**

#Part 1

For this assignment, it is required to explore and cluster the neighborhoods in Toronto.

1. First, a specific notebook is created.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


3. To create the above dataframe:

  The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)



In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### 1. Scrapping the given Wikipedia link

In [0]:
wikip = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050." #I had to use an old version of the page because the table format changed completely
fuente = requests.get(wikip).text

In [0]:
soup = BeautifulSoup(fuente, 'xml')
table=soup.find('table')

In [0]:
nombres_columnas = ['Postalcode','Borough','Neighborhood']
tabla = pd.DataFrame(columns = nombres_columnas)


In [0]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        tabla.loc[len(tabla)] = row_data

In [42]:
tabla.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###Data Wrangling

Next step is to remove "Not assigned" elements

In [0]:
tabla=tabla[tabla['Borough']!='Not assigned']

In [44]:
tabla=tabla[tabla.Borough != 'Not assigned']
tabla = tabla.sort_values(by=['Postalcode','Borough'])

tabla.reset_index(inplace=True)
tabla.drop('index',axis=1,inplace=True)

tabla.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


###2. Merging neighbourhoods

Neighbourhoods are grouped based on the PostalCode.

In [0]:
tabla2=tabla.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
tabla2=tabla2.reset_index(drop=False)
tabla2.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [0]:
tabla3 = pd.merge(tabla, tabla2, on='Postalcode')

In [0]:
tabla3.drop(['Neighborhood'],axis=1,inplace=True)

In [48]:
tabla3.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
tabla3.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"


In [49]:
tabla3.shape #We get the shape of the table tabla3

(210, 3)

#Part 2

Now a dataframe of the postal code of each neighborhood is built, along with the borough name and neighborhood name, in order to utilize the Foursquare location data, it is needed to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, leveraging the Google Maps Geocoding API was used to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so I will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

UPDATE: Geocoder did not work. I used the given csv file instead.

In [50]:
import csv
tabla_coordenadas = pd.read_csv("https://cocl.us/Geospatial_data")
tabla_coordenadas.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
tabla_coord = tabla_coordenadas.rename({'Postal Code':'Postalcode'}, axis=1)

# Merge coordinates into neighbourhood dataframe
tabla_Toronto = tabla3.merge(tabla_coord)
tabla_Toronto

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
...,...,...,...,...,...
205,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
206,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
207,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
208,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


Now, a DataFrame with the Neighbourhood and coordinates data is available. This is the end of the Segmenting and Clustering  Neighbourhoods in Toronto task.